In [2]:
# import pdb; pdb.set_trace()
# import pudb; pudb.set_trace()

# df.style.background_gradient(
#     cmap='coolwarm', 
#     axis=None, 
#     vmin=-1, 
#     vmax=1
# )  # 'Blues', 'coolwarm', 'RdBu_r' & 'BrBG'

# df.style.set_sticky(axis="index")

# pd.set_option('precision', 2)

# from pandas_profiling import ProfileReport
# profile = ProfileReport(df, title="Pandas Profiling Report", explorative=True)

# .plot_bokeh(sizing_mode="scale_height")
# .plot_bokeh(kind='hist', bins=1000, sizing_mode="scale_height", xlim=[0, .1])

# Imports

In [3]:
%load_ext autoreload
%autoreload 2

import pandas as pd
pd.set_option('float_format', '{:f}'.format)
pd.set_option('display.max_rows', 40)
pd.set_option('display.max_columns', 99)

import matplotlib as mpl
from matplotlib import style
style.use('fivethirtyeight')
mpl.rcParams['figure.figsize'] = [20, 7]

import datetime as dt
import pickle
import numpy as np
import pandas as pd
import pandas_bokeh
pandas_bokeh.output_file("tmp/bokeh_output.html")
import ray

from data_layer import storage_adaptor, fsspec_factory, data_access, arrow_dataset
from tick_filter import streaming_tick_filter
from tick_sampler import streaming_tick_sampler, daily_stats
from utilities import date_fu, project_globals as g
from workflows import sampler_task, sampler_flow, syncbar_flow, syncbar_task
from workflows.configs import renko_v1, renko_v2, renko_v3
from sample_features import stacked
from filters import jma

In [4]:
config = renko_v2.config

prefix_data = '/data/trades'

prefix_meta = f"/bars/{config['meta']['config_id']}/meta"

prefix_df = f"/bars/{config['meta']['config_id']}/df"

prefix_sb = f"/bars/{config['meta']['config_id']}/sync_bars/clock_symbol=MUX"

In [5]:
df = data_access.fetch_sd_data('AEM', prefix=prefix_df, date='2019-01-03')

In [6]:
sum(abs(df.label_rrr)>=3) / df.shape[0]

0.38

In [ ]:
abs(df.label_rrr).hist(bins=30)

# load multi-file dataset

In [4]:
ds = arrow_dataset.get_dataset('/bars/renko_v2/df')

df = ds.to_table().to_pandas()

sdf = df[df.symbol=='SAND']

In [7]:
ds = arrow_dataset.get_dataset('/bars/renko_v2/sync_bars')

df2 = ds.to_table().to_pandas()

sdf2 = df2[(df2.clock_symbol=='IAG') & (df2.symbol=='GLD')].reset_index(drop=True)

# Price gap fill

In [22]:
fdf = stacked.fill_gaps_dates(sdf, 'price_vwap')

In [23]:
fdf[['price_vwap','price_high','price_low']].plot_bokeh(sizing_mode="scale_height")

Figure(id='1725', ...)

# JMA filter

In [ ]:
jma.jma_expanding_filter(fdf.price_vwap, winlen=200, power=1, phase=0)

In [ ]:
jma.jma_filter_df(fdf, 'price_vwap', winlen=205, power=1)

In [ ]:
jma.jma_filter_df(fdf, 'price_vwap', winlen=2000, power=1).price_vwap_jma.plot()

In [ ]:
jma.jma_filter_df(fdf, 'price_vwap', winlen=1000, power=1).price_vwap_jma.diff().hist(bins=40)

In [ ]:
fdf.date.value_counts().hist(bins=40)

# Ray setup

In [ ]:
ray.init(dashboard_host='0.0.0.0', dashboard_port=1111, ignore_reinit_error=True)

In [ ]:
ray.shutdown()

In [ ]:
all_symbols = data_access.list_sd_data(prefix='/bars/renko_v2/df', source='remote')

In [ ]:
large_caps, mid_caps = syncbar_task.segment_symbols(
    all_symbols, start_date='2019-01-01', end_date='2021-01-01'
)

In [ ]:
large_caps

In [ ]:
mid_caps

# sync bars

In [ ]:
config = {
    'config_id': 'renko_v2',
    'start_date': '2019-01-01',
    'end_date': '2021-01-01',
    'source': 'remote',
    'destination': 'both',
    'on_ray': True,
}

In [ ]:
syncbar_flow.run(config)

# Tick Sampler 

In [ ]:
bds = sampler_flow.run(config)

# fracdiff test

In [ ]:
import statsmodels.tsa.stattools as stattools
import fracdiff

X = bdf.price_close.values.reshape(-1, 1)

fs = fracdiff.FracdiffStat(mode="valid")

Xdiff = fs.fit_transform(X)

_, pvalue, _, _, _, _ = stattools.adfuller(Xdiff)

corr = np.corrcoef(X[-Xdiff.size :, 0], Xdiff.reshape(-1))[0][1]

print("* Order: {:.2f}".format(fs.d_[0]))
print("* ADF p-value: {:.2f} %".format(100 * pvalue))
print("* Correlation with the original time-series: {:.2f}".format(corr))

spx = bdf.price_close

import matplotlib.pyplot as plt

spx_diff = pd.Series(Xdiff.reshape(-1), index=spx.index[-Xdiff.size :])

fig, ax_s = plt.subplots(figsize=(24, 6))
plt.title("S&P 500 and its differentiation preserving memory")
ax_d = ax_s.twinx()

plot_s = ax_s.plot(spx, color="blue", linewidth=4, label="S&P 500 (left)")
plot_d = ax_d.plot(
    spx_diff,
    color="orange",
    linewidth=4,
    label=f"S&P 500, {fs.d_[0]:.2f} th diff (right)",
)
plots = plot_s + plot_d

ax_s.legend(plots, [p.get_label() for p in plots], loc=0)
plt.show()